In [2]:
import spacy
import csv
import re
from collections import OrderedDict
from typing import Dict, List, Tuple, Optional

# Constants
DEFAULT_PRONOUNS = {
    "male": {"subject": "he", "object": "him", "possessive": "his", "reflexive": "himself"},
    "female": {"subject": "she", "object": "her", "possessive": "her", "reflexive": "herself"},
    "gender_fair": {"subject": "they", "object": "them", "possessive": "their", "reflexive": "themselves"}
}

GENDER_ADJECTIVES = {"male", "female", "lady", "gentlemen", "boy", "girl", "man", "woman"}
GENDER_PAIRS = {
    ("girl", "boy"): ["children", "kids", "students", "youth", "young people"],
    ("son", "daughter"): ["children", "kids", "offspring"],
    ("woman", "man"): ["people", "individuals", "persons"],
    ("women", "men"): ["people", "individuals", "persons"],
    ("he", "she"): ["they"],
    ("his", "her"): ["their"],
    ("him", "her"): ["them"],
    ("himself", "herself"): ["themselves"],
    ("husband", "wife"): ["spouse", "partner"],
    ("boyfriend", "girlfriend"): ["partner"],
    ("brother", "sister"): ["sibling"],
    ("father", "mother"): ["parent"],
    ("uncle", "aunt"): ["relative"],
    ("nephew", "niece"): ["relative"],
    ("ladies", "gentlemen"): ["everyone", "all"]
}

nlp = spacy.load('en_core_web_sm')

class GenderFairLanguage:
    def __init__(self, terms_csv: str = 'gendered_terms.csv'):
        self.gendered_terms = self._load_and_prioritize_terms(terms_csv)
        
    def _load_and_prioritize_terms(self, csv_filename: str) -> OrderedDict:
        """Load and prioritize gendered terms from CSV with multiple replacements."""
        gendered_terms = OrderedDict()
        try:
            with open(csv_filename, 'r') as csvfile:
                reader = csv.reader(csvfile)
                for row in reader:
                    if len(row) >= 2:
                        term = row[0].strip().lower()
                        replacements = [repl.strip() for repl in row[1:] if repl.strip()]
                        if term and replacements:
                            gendered_terms[term] = replacements
        except Exception as e:
            raise ValueError(f"Error loading gendered terms: {e}")
            
        return OrderedDict(
            sorted(gendered_terms.items(), key=lambda item: len(item[0].split()), reverse=True)
        )

    @staticmethod
    def _adjust_pluralization(adjective: str, noun: str) -> str:
        """Proper static method implementation for plural adjustment."""
        adj_lower = adjective.lower()
        noun_lower = noun.lower()
        
        # Handle irregular plurals
        irregular_plurals = {
            "children": "child",
            "people": "person",
            "women": "woman",
            "men": "man"
        }
        
        # Check if noun is plural
        is_noun_plural = (
            noun_lower.endswith('s') or 
            noun_lower in irregular_plurals.values()
        )
        
        # Check if adjective is plural
        is_adj_plural = (
            adj_lower.endswith('s') or 
            adj_lower in irregular_plurals
        )
        
        # Maintain plural consistency
        if is_adj_plural and not is_noun_plural:
            return f"{noun}s" if not noun.endswith('s') else noun
        elif not is_adj_plural and is_noun_plural:
            return noun[:-1] if noun.endswith('s') else noun
        return noun
    
    @staticmethod
    def _adjust_capitalization(original: str, replacement: str) -> str:
        """Preserve capitalization patterns in replacements."""
        if original.isupper():
            return replacement.upper()
        elif original.istitle():
            return replacement.capitalize()
        return replacement.lower()

    def _is_within_quotes(self, doc, start_idx: int, end_idx: int) -> bool:
        """Check if text is within double quotes in the original text."""
        text = doc.text
        before = len(re.findall(r'(?<!\\)"', text[:start_idx]))
        after = len(re.findall(r'(?<!\\)"', text[end_idx:]))
        return (before % 2 == 1) and (after % 2 == 1)

    def _process_text_replacements(self, text: str, name_pronoun_map: Dict) -> Tuple[str, List[Dict]]:
        """Main text processing with hyphen-aware name handling."""
        doc = nlp(text)
        # Enhanced title detection with hyphen handling

        TITLE_MAPPING = {
                        'mr': 'male',
                        'ms': 'female', 
                        'mrs': 'female', 
                        'mx': 'gender_fair',
                        'sir': 'male',
                        'madam': "female",
                         }
        i = 0
        while i < len(doc):
            token = doc[i]
            base_title = token.text.lower().rstrip('.')

            if base_title in TITLE_MAPPING:
                name_parts = [token.text]
                j = i + 1
                hyphen_buffer = None  # Track hyphens for merging

                while j < len(doc):
                    current_token = doc[j]
                    current_text = current_token.text

                    # Handle hyphen merging
                    if current_text == '-':
                        hyphen_buffer = current_text
                    elif hyphen_buffer and current_token.pos_ == 'PROPN':
                        # Merge hyphen with next word
                        name_parts.append(hyphen_buffer + current_text)
                        hyphen_buffer = None
                    elif current_token.pos_ == 'PROPN' or (current_text in ['-'] and not hyphen_buffer):
                        name_parts.append(current_text)
                    else:
                        break
                    
                    j += 1

                if len(name_parts) > 1:
                    # Clean and normalize names
                    full_name = self._normalize_hyphenated_name(' '.join(name_parts))
                    name_without_title = self._normalize_hyphenated_name(' '.join(name_parts[1:]))

                    category = TITLE_MAPPING[base_title]
                    name_pronoun_map[full_name.lower()] = category
                    name_pronoun_map[name_without_title.lower()] = category

                i = j - 1  # Skip processed tokens
            i += 1

        # Rest of processing remains
        corrections = []
        corrections.extend(self._find_gender_pairs(text, doc))
        corrections.extend(self._find_adjective_noun_pairs(text, doc))
        corrections.extend(self._find_individual_terms(text, doc))
        corrections.extend(self._find_pronoun_replacements(text, doc, name_pronoun_map))
        corrections = self._filter_overlapping_corrections(corrections)

        revised_text = text
        for correction in sorted(corrections, key=lambda x: -x['character_offset']):
            replacement_str = correction['replacements'][0]
            revised_text = revised_text[:correction['character_offset']] + replacement_str + revised_text[correction['character_endset']:]

        return revised_text, corrections

    @staticmethod
    def _normalize_hyphenated_name(name: str) -> str:
        """Merge hyphen patterns into proper hyphenated format."""
        return re.sub(r'\s*-\s*', '-', name).strip()

    def _find_pronoun_replacements(self, text: str, doc, name_pronoun_map: Dict) -> List[Dict]:
        """Pronoun resolution with enhanced name tracking."""
        corrections = []
        name_to_category = {name.lower(): category for name, category in name_pronoun_map.items()}
        processed_names = set(name_to_category.keys())

        # Track all name variants from titles
        name_variants = []
        for name in processed_names:
            name_variants.extend([
                name,
                name.replace('-', ' '),  # "adam-silver" -> "adam silver"
                name.replace(' ', '-')    # "adam silver" -> "adam-silver"
            ])

        NEUTRAL_PRONOUNS = {"i", "me", "my", "mine", "myself",
                            "we", "us", "our", "ours", "ourselves",
                            "you", "your", "yours", "yourself", "yourselves",
                            "they", "them", "their", "theirs", "themselves", "it", "its", "itself"}

        for token in doc:
            if (token.tag_ in ["PRP", "PRP$"] and 
                token.text.lower() not in NEUTRAL_PRONOUNS and
                not self._is_within_quotes(doc, token.idx, token.idx + len(token.text))):

                # Find closest matching name in previous context
                referent = None
                lookback_window = doc[max(0, token.i-10):token.i]  # 10 tokens back
                for prev_token in reversed(lookback_window):
                    if prev_token.pos_ == 'PROPN':
                        candidate = self._normalize_hyphenated_name(prev_token.text).lower()
                        if candidate in processed_names:
                            referent = candidate
                            break
                        # Check space/hyphen variants
                        for variant in [candidate, candidate.replace('-', ' ')]:
                            if variant in name_to_category:
                                referent = variant
                                break
                        if referent:
                            break

                # Fallback to entity resolution
                if not referent:
                    for ent in reversed(doc.ents):
                        if ent.end <= token.i and ent.label_ in ["PERSON", "ORG"]:
                            candidate = self._normalize_hyphenated_name(ent.text).lower()
                            if candidate in processed_names:
                                referent = candidate
                                break

                # Determine pronoun role and replacement
                if referent:
                    category = name_to_category[referent]
                    role = self._get_pronoun_role(token)
                    new_pronoun = DEFAULT_PRONOUNS[category][role]
                else:
                    new_pronoun = DEFAULT_PRONOUNS["gender_fair"][self._get_pronoun_role(token)]

                # Apply capitalization and replacement
                new_pronoun = self._adjust_capitalization(token.text, new_pronoun)
                if new_pronoun != token.text:
                    corrections.append({
                        "word_index": token.i,
                        "original_text": token.text,
                        "replacements": [new_pronoun],
                        "character_offset": token.idx,
                        "character_endset": token.idx + len(token.text)
                    })

        return corrections

    def _get_pronoun_role(self, token) -> str:
        """Determine pronoun grammatical role."""
        if token.dep_ in ["nsubj", "nsubjpass"]:
            return "subject"
        if token.dep_ in ["dobj", "iobj", "pobj"]:
            return "object"
        if token.dep_ == "poss" or token.tag_ == "PRP$":
            return "possessive"
        if token.dep_ == "reflexive":
            return "reflexive"
        return "subject"  # Default fallback

    def _find_gender_pairs(self, text: str, doc) -> List[Dict]:
        """Find specific gender pairs to replace with more inclusive terms."""
        corrections = []

        for (term1, term2), replacements in GENDER_PAIRS.items():
             for t1, t2 in [(term1, term2), (term2, term1)]:
                term1_variants = [t1, t1 + 's', t1 + 'es']
                term2_variants = [t2, t2 + 's', t2 + 'es']

                for v1 in term1_variants:
                    for v2 in term2_variants:
                        pattern = rf'\b({v1})\s+(and|or)\s+({v2})\b'
                        matches = list(re.finditer(pattern, text, re.IGNORECASE))

                        for match in reversed(matches):
                            if self._is_within_quotes(doc, match.start(), match.end()):
                                continue

                            is_or = match.group(2).lower() == "or"

                            # Handle singular/plural for "or" cases
                            if is_or:
                                adjusted_replacements = []
                                for repl in replacements:
                                    if repl.endswith('s'):
                                        adjusted = repl[:-1]  # children -> child
                                    elif repl.endswith('es'):
                                        adjusted = repl[:-2]  # ladies -> lady
                                    else:
                                        adjusted = repl
                                    adjusted_replacements.append(adjusted)
                            else:
                                adjusted_replacements = replacements

                            first_word = match.group(1)
                            adjusted_replacements = [
                                self._adjust_capitalization(first_word, repl)
                                for repl in adjusted_replacements
                            ]

                            corrections.append({
                                "word_index": next((i for i, t in enumerate(doc) if t.idx <= match.start() < t.idx + len(t.text)), None),
                                "original_text": match.group(0),
                                "replacements": adjusted_replacements,
                                "character_offset": match.start(),
                                "character_endset": match.end()
                            })

        return corrections


    def _find_adjective_noun_pairs(self, text: str, doc) -> List[Dict]:
        """Enhanced adjective-noun pair handling with proper static method calls."""
        corrections = []
        GENDER_ADJECTIVES_PAIR_REMOVAL = {"male", "female", "lady", "gentlemen", "boy", "girl", "lady's"}

        for i in range(len(doc) - 1):
            token = doc[i]
            next_token = doc[i + 1]

            # Process adjective removal pairs
            if (token.text.lower() in GENDER_ADJECTIVES_PAIR_REMOVAL and 
                next_token.pos_ == "NOUN"):

                # Get replacement with proper pluralization
                replacement = self._adjust_pluralization(
                    adjective=token.text,
                    noun=next_token.text
                )

                # Create correction entry
                corrections.append({
                    "word_index": i,
                    "original_text": f"{token.text} {next_token.text}",
                    "replacements": [replacement],
                    "character_offset": token.idx,
                    "character_endset": next_token.idx + len(next_token.text)
                })


            elif (token.text.lower() in GENDER_ADJECTIVES and 
                next_token.pos_ == "NOUN" and
                not self._is_within_quotes(doc, token.idx, next_token.idx + len(next_token.text))):

                noun_replacements = self.gendered_terms.get(next_token.text.lower(), [next_token.text])
                adjusted_replacements = [
                    self._adjust_capitalization(next_token.text, repl)
                    for repl in noun_replacements
                ]

                corrections.append({
                    "word_index": i,
                    "original_text": f"{token.text} {next_token.text}",
                    "replacements": adjusted_replacements,
                    "character_offset": token.idx,
                    "character_endset": next_token.idx + len(next_token.text)
                })
    
        return corrections


    def _find_individual_terms(self, text: str, doc) -> List[Dict]:
        """Find individual gendered terms."""
        corrections = []
        
        for term, replacements_list in self.gendered_terms.items():
            if ' ' in term:
                continue
            
            for match in reversed(list(re.finditer(rf'\b{re.escape(term)}\b', text, re.IGNORECASE))):
                if self._is_within_quotes(doc, match.start(), match.end()):
                    continue
                
                original_text = match.group(0)
                adjusted_replacements = [self._adjust_capitalization(original_text, repl) for repl in replacements_list]
                
                corrections.append({
                    "word_index": next((i for i, t in enumerate(doc) if t.idx <= match.start() < t.idx + len(t.text)), None),
                    "original_text": original_text,
                    "replacements": adjusted_replacements,
                    "character_offset": match.start(),
                    "character_endset": match.end()
                })
        
        return corrections

    def _find_pronoun_replacements(self, text: str, doc, name_pronoun_map: Dict) -> List[Dict]:
        """Find pronoun replacements with fallback to gender-fair language."""
        corrections = []
        name_to_category = {name.lower(): category for name, category in (name_pronoun_map or {}).items()}

        person_entities = [ent for ent in doc.ents if ent.label_ in ["PERSON", "ORG"]]
        NEUTRAL_PRONOUNS = {"i", "me", "my", "mine", "myself",
                            "we", "us", "our", "ours", "ourselves",
                            "you", "your", "yours", "yourself", "yourselves",
                            "they", "them", "their", "theirs", "themselves", "it", "its", "itself"}

        for token in doc:
            if (token.tag_ in ["PRP", "PRP$"] and 
                token.text.lower() not in NEUTRAL_PRONOUNS and
                not self._is_within_quotes(doc, token.idx, token.idx + len(token.text))):

                referent = None
                for ent in reversed(person_entities[:token.i]):
                    if ent.end <= token.i:
                        referent = ent.text
                        break
                
                if token.dep_ in ["nsubj", "nsubjpass"]:
                    role = "subject"
                elif token.dep_ in ["dobj", "iobj", "pobj"]:
                    role = "object"
                elif token.dep_ == "poss" or token.tag_ == "PRP$":
                    role = "possessive"
                elif token.dep_ == "reflexive":
                    role = "reflexive"
                else:
                    continue
                
                if referent and referent.lower() in name_to_category:
                    category = name_to_category[referent.lower()]
                    new_pronoun = DEFAULT_PRONOUNS[category][role]
                else:
                    new_pronoun = DEFAULT_PRONOUNS["gender_fair"][role]

                if token.text.istitle():
                    new_pronoun = new_pronoun.capitalize()
                elif token.text.isupper():
                    new_pronoun = new_pronoun.upper()

                if new_pronoun.lower() != token.text.lower():
                    corrections.append({
                        "word_index": token.i,
                        "original_text": token.text,
                        "replacements": [new_pronoun],
                        "character_offset": token.idx,
                        "character_endset": token.idx + len(token.text)
                    })

        return corrections

    def _filter_overlapping_corrections(self, corrections: List[Dict]) -> List[Dict]:
        """Filter out overlapping corrections, keeping the longest matches."""
        if not corrections:
            return []
            
        sorted_corrections = sorted(corrections, key=lambda x: (x['character_offset'], -x['character_endset']))
        filtered = []
        last_end = -1
        
        for c in sorted_corrections:
            if c['character_offset'] >= last_end:
                filtered.append(c)
                last_end = c['character_endset']
        
        return filtered

    def process_text(self, text: str, name_pronoun_map: Optional[Dict] = None) -> Dict:
        """Main processing method that returns original and revised text with corrections."""
        if not text or not isinstance(text, str):
            raise ValueError("Input text must be a non-empty string")
            
        revised_text, corrections = self._process_text_replacements(text, name_pronoun_map or {})
        
        return {
            "original_text": text,
            "revised_text": revised_text,
            "corrections": sorted([
                c for c in corrections if c['character_offset'] >= 0
            ], key=lambda x: x['character_offset'])
        }

def load_gfl(csv_path: str = 'gendered_terms.csv') -> GenderFairLanguage:
    return GenderFairLanguage(csv_path)


In [3]:
gfl = load_gfl()
 
# Example usage
input_text = "Mr. adam silver is businessperson"
pronoun_map =  { "Adam Silver" : "gender_fair" }

 
result = gfl.process_text(input_text, pronoun_map)
import json
print(json.dumps(result, indent=2))

{
  "original_text": "Mr. adam silver is businessperson",
  "revised_text": "Mr. adam silver is businessperson",
  "corrections": [
    {
      "word_index": 4,
      "original_text": "businessperson",
      "replacements": [
        "businessperson"
      ],
      "character_offset": 19,
      "character_endset": 33
    }
  ]
}
